In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
 
from sklearn.model_selection import train_test_split
from sklearn import metrics
 
import cv2
import gc
import os
 
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Sequential
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds_store_file_location = 'directory/.DS_store'
if os.path.isfile(ds_store_file_location):
    os.remove(ds_store_file_location)

In [ ]:
path = '/Volumes/ExternalHD/AD_dataset'
classes = ['MildDemented','ModerateDemented','VeryMildDemented','NonDemented']

In [ ]:
for cat in classes:
    image_dir = os.path.join(path, cat)
    images = [img for img in os.listdir(image_dir) if img.lower().endswith(('.jpg')) and not img.startswith('.')]
    
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'Images for {cat} category', fontsize=20)
    for i in range(3):
            k = np.random.randint(0, len(images))
            img_path = os.path.join(image_dir, images[k])
            img = np.array(Image.open(img_path))
            ax[i].imshow(img)
            ax[i].axis('off')
    plt.show()

In [ ]:
X = []
Y = []

for i, cat in enumerate(classes):
    images = glob(f'{path}/{cat}/*.jpg')

    for image in images:
        img = cv2.imread(image)
        resized_img = cv2.resize(img, (128, 128))
        X.append(resized_img)
        Y.append(i)
        
X = np.asarray(X)
one_hot_encoded_Y = pd.get_dummies(Y).values

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, one_hot_encoded_Y, test_size = 0.2, 
                                                  random_state = 1901)
print(X_train.shape, X_val.shape)

In [ ]:
# LeNet-5 CNN Architecture
model = Sequential()
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(128,128,3)))
model.add(layers.AveragePooling2D(pool_size=(2, 2))) 
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D(pool_size=(2, 2))) 
model.add(layers.Flatten())
model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))
model.add(layers.Dense(units=4, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes = True, show_dtype = True, show_layer_activations = True)


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
from keras.callbacks import ReduceLROnPlateau
def on_epoch_end(self, epoch, logs={}):
    pass
lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1)

In [ ]:
history = model.fit(X_train, 
                    Y_train, 
                    validation_data = (X_val, Y_val), 
                    batch_size = 64, 
                    epochs = 10, 
                    verbose = 1, 
                    callbacks = [lr, myCallback()])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()
history_df.loc[:,['accuracy','val_accuracy']].plot()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_val, Y_val, verbose=2)
print(test_acc)

In [ ]:
Y_pred = model.predict(X_val)
Y_val = np.argmax(Y_val, axis=1)
Y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print(metrics.classification_report(Y_val, Y_pred, target_names=classes))

In [ ]:
#skip cell

Y_probs = model.predict_proba(X_val)
avg_probs = np.mean(Y_probs, axis=0)
plt.figure(figsize=(10, 6))
plt.bar(classes, avg_probs, color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Average Probability')
plt.title('Average Prediction Probabilities')
plt.show()

In [ ]:
pred_prob = model.predict(X_val)
avg_prob = np.mean(pred_prob, axis=0)
plt.figure(figsize=(10, 6))
plt.bar(classes, avg_prob)
plt.xlabel('Classes')
plt.ylabel('Probability')
plt.title('Average Probability Distribution Across All Predictions')
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
c_matrix = confusion_matrix(Y_val, Y_pred)

plt.figure(figsize=(7, 6))
sns.heatmap(c_matrix, cmap='Blues', annot=True, fmt='d', xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()